In [161]:
%load_ext autoreload
%autoreload
import time, datetime
import os
import pandas as pd

from utils import get_volumeKospiHistory, get_volumeKosdaqHistory, get_priceHistory

pd.set_option('display.max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
path = "./Kospi/"
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
for v in file_list_csv:
    print(v[:6] + v[7:-4])

000120CJ대한통운
139480이마트
086790하나금융지주
007160사조산업
003670포스코케미칼
084680이월드
001680대상
009460한창제지
009180한솔로지스틱스
133820화인베스틸
002420세기상사
016580환인제약
298040효성중공업
006340대원전선
003410쌍용양회
024070WISCOM
002450삼익악기
117580대성에너지
152330코리아오토글라스
012280영화금속
011280태림포장
000860강남제비스코
003580넥스트사이언스
267290경동도시가스
033920무학
249420일동제약
004080신흥
016610DB금융투자
007980태평양물산
092230KPX홀딩스
004270남성
012170키위미디어그룹
021820세원정공
005420코스모화학
009420한올바이오파마
014280금강공업
020150일진머티리얼즈
194370제이에스코퍼레이션
000760이화산업
016380KG동부제철
082640동양생명
009320대우부품
322000현대에너지솔루션
170900동아에스티
001460BYC
003680한성기업
248170샘표식품
008040사조동아원
004100태양금속
000660SK하이닉스
002840미원상사
032640LG유플러스
015020이스타코
004990롯데지주
005610SPC삼ᄅ

In [219]:
path = "./Kosdaq/"
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
for v in file_list_csv:
    print(v[:6] + v[7:-4])

160980싸이맥스
290120대유에이피
050890쏠리드
150840인트로메딕
075130플랜티넷
182360큐브엔터
036010아비코전자
012790신일제약
031310아이즈비전
238490힘스
036030KTH
200780비씨월드제약
193250와이제이엠게임즈
267790배럴
034950한국기업평가
142210유니트론텍
307070SK4호스팩
032080아즈텍WB
131100스카이이앤엠
080580오킨스전자
044480경남바이오파마
033560블루콤
033170시그네틱스
070300엑스큐어
109960에이프로젠 H&G
069920아이에스이커머스
221980케이디켐
039560다산네트웍스
042500링네트
026150특수건설
100590머큐리
045300성우테크론
023760한국캐피탈
025980아난티
222110팬젠
123860아나패스
089600나스미디어
065680우주일렉트로
119860다나와
121850코이즈
047080한빛소프트
290510코리아센터
047560이스트소프트
007390네이처셀
238200비피도
340120하이제5호스팩
011370서한
194700노바렉스
206560덱스터
050090휘닉스소재
065560녹원씨엔아이
067570엔브이에이치코리아
204630플리트 엔터테인먼트
270870뉴트리
065620제낙스
045100한양이엔지
013310아진산어

In [197]:
def getKospiMergedTab(code, name, start_date, end_date):
    vol = get_volumeKospiHistory(code, name, start_date, end_date).fillna(0)
    p = get_priceHistory(code,name, start_date, end_date).fillna(0)
    df = pd.concat([vol, p], axis=1, join='inner')
    
    return df

df008500merged = getKospiMergedTab(8500, "일정실업", '2010-01-01', '2019-12-31')

In [198]:
def accumConsecutiveCount(df):
    df['accum'] = (df['NetBuyAmount_Inst'] > 0).astype(int)
    last_index = df.index[0]
    init_index = last_index + pd.DateOffset(1)
    for index, row in df[init_index:].iterrows():
        if df.loc[index].accum > 0:
            df.loc[index,'accum'] = df.loc[last_index, 'accum'] + 1
        last_index = index
    return df

df008500 = accumConsecutiveCount(df008500merged)

In [218]:
def computeRevenue(df):
    output_df = pd.DataFrame(columns=['매수일', '6th', '7th','8th','9th','10th','11th','12th','13th','14th','15th'])
    for i, _ in df[df['accum']>=5].iterrows():
        tmp_v = list()
        next_day = df.index.get_loc(i) + 1
        base_day = df.iloc[next_day-6].at['date'][0]
        tmp_v.append(base_day)
        base_price = df.iloc[next_day]['Open']
        for _ in range(10):
            close_price = df.iloc[next_day]['Close']
            revenue = ((close_price - base_price) * 100) / base_price
            tmp_v.append(revenue)
            next_day = next_day + 1
        tmp_series = pd.Series(tmp_v, index = output_df.columns)
        output_df = output_df.append(tmp_series, ignore_index=True)
    return output_df
df = computeRevenue(df008500)
code = "008500"
name = "일정실업"

file_path = './output/raw_' + code + "_" + name + ".csv"
df.to_csv(file_path, index = False, header=True)

In [216]:
def getSummary(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','항목', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    tmpList.append('MAX')
    for i in range(6, 16, 1):
        df_min = df[str(i)+'th'].max()
        tmpList.append(df_min)
    tmp_series = pd.Series(tmpList, index = output_df.columns)
    output_df = output_df.append(tmp_series, ignore_index=True)

    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    tmpList.append('MIN')
    for i in range(6, 16, 1):
        df_max = df[str(i)+'th'].min()
        tmpList.append(df_max)
    tmp_series = pd.Series(tmpList, index = output_df.columns)
    output_df = output_df.append(tmp_series, ignore_index=True)

    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    tmpList.append('Mean')
    for i in range(6, 16, 1):
        df_mean = df[str(i)+'th'].mean()
        tmpList.append(df_mean)
    tmp_series = pd.Series(tmpList, index = output_df.columns)
    output_df = output_df.append(tmp_series, ignore_index=True)

    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    tmpList.append('Median')
    for i in range(6, 16, 1):
        df_median = df[str(i)+'th'].median()
        tmpList.append(df_median)
    tmp_series = pd.Series(tmpList, index = output_df.columns)
    output_df = output_df.append(tmp_series, ignore_index=True)

    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    tmpList.append('Probability')
    for i in range(6, 16, 1):
        df_prob = df[df[str(i)+'th']>0].shape[0]/df.shape[0]
        tmpList.append(df_prob)
    tmp_series = pd.Series(tmpList, index = output_df.columns)
    output_df = output_df.append(tmp_series, ignore_index=True)

    return output_df
    
code = "008500"
name = "일정실업"

output_df = getSummary(code, name, df)

In [217]:
file_path = './output/summary_' + str(code) + "_" + name + ".csv"
output_df.to_csv(file_path, index = False, header=True)